# Zadatak: Maksimizacija zarade u pekari

Jedna pekara proizvodi dve vrste proizvoda: **kifle** i **kroasane**.

- Kifla donosi **24 dinara** profita po komadu, a kroasan **33 dinara** po komadu.  
- Pekara želi da proizvede **najmanje 100 kifli** i **najmanje 80 kroasana** dnevno.  
- Za proizvodnju jedne kifle potrebno je **100 grama brašna**, a za jedan kroasan **150 grama brašna**.  
- Ukupna količina brašna dostupna u pekari dnevno je **32 kg** (tj. 32_000 grama).

**Zadatak:** Odrediti optimalnu dnevnu proizvodnju kifli i kroasana tako da se **maksimizuje ukupna zarada**, uz poštovanje minimalnih količina i ograničenja po brašnu.

## Matematički model

Označimo:  
- $x_1$ = broj kifli  
- $x_2$ = broj kroasana  

**Funkcija cilja (maksimizacija profita):**  
$$
\text{max } Z = 24 x_1 + 33 x_2
$$

**Ograničenja:**  
$$
\begin{cases}
x_1 \ge 100 \\
x_2 \ge 80 \\
100 x_1 + 150 x_2 \le 32000
\end{cases}
$$

Zadatak možemo rešiti linearnim programiranjem koristeći, na primer, `scipy.optimize.linprog`.


In [1]:
from scipy.optimize import linprog

In [2]:
linprog(c = [-24, -33], A_ub = [[-1, 0], [0, -1], [100, 150]], b_ub = [-100, -80, 32000])

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -7440.0
              x: [ 2.000e+02  8.000e+01]
            nit: 0
          lower:  residual: [ 2.000e+02  8.000e+01]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.000e+02  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -3.000e+00 -2.400e-01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

# Uncapacitated Facility Location Problem (UFLP)

**Opis problema:**

Uncapacitated Facility Location Problem (UFLP) je klasičan problem optimizacije u oblasti logistike i operacionih istraživanja. Problem se može opisati ovako:

- Imamo skup potencijalnih **resursa** i skup **korisnika**.  
- Svaka lokacija resursa ima **fiksni trošak izgradnje**.  
- Svaki korisnik ima **potrebu** koja može biti zadovoljena iz bilo kojeg otvorenog resursa, pri čemu postoji **trošak transporta** između resursa i korisnika.  
- Cilj je odabrati koje resurse otvoriti i kako dodeliti korisnike tim resursima tako da **ukupni trošak (fiksna cena + trošak transporta) bude minimalan**.

**Formalna definicija:**

Označimo:  
- $R$ = skup svih potencijalnih resursa  
- $U$ = skup svih korisnika  
- $z_i$ = fiksna cena otvaranja resursa $i \in R$  
- $c_{ij}$ = trošak transporta korisnika $j \in U$ do resursa $i \in R$  
- $x_i \in \{0,1\}$ = 1 ako je resurs $i$ izgrađen, 0 inače  
- $y_{ij} \in \{0,1\}$ = 1 ako korisnik $j$ koristi resurs $i$, 0 inače  

**Ciljna funkcija:**
$$
\min \sum_{i \in R} z_i x_i + \sum_{i \in R} \sum_{j \in U} c_{ij} y_{ij}
$$

**Ograničenja:**
1. Svaki korisnik mora biti dodeljen tačno jednom resursu:
$$
\sum_{i \in R} y_{ij} = 1 \quad \forall j \in U
$$
2. Korisnik može biti dodeljen samo otvorenom resursu:
$$
y_{ij} \le x_i \quad \forall i \in R, \forall j \in U
$$
3. Promenljive su binarne:
$$
x_i \in \{0,1\}, \quad y_{ij} \in \{0,1\}
$$


In [3]:
from uflp_utils import read_instance, read_bk_instance

In [4]:
c, z = read_instance('../2022_2023/live/03_s_metaheuristics/uflp1.txt')
# c, z = read_bk_instance('BildeKrarup/B/B1.1')

In [5]:
c, z

([[1, 12, 3], [2, 7, 41], [19, 21, 7]], [12, 11, 13])

In [6]:
num_resources = len(z)
num_users = len(c)

In [7]:
from docplex.mp.model import Model

In [8]:
model = Model(name='uflp')

In [9]:
x = model.binary_var_list(num_resources, name='x')
x

[docplex.mp.Var(type=B,name='x_0'),
 docplex.mp.Var(type=B,name='x_1'),
 docplex.mp.Var(type=B,name='x_2')]

In [10]:
y = model.binary_var_matrix(num_users, num_resources, name='y')


In [11]:
model.print_information()

Model: uflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [12]:
for u in range(num_users):
    for i in range(num_resources):
        model.add_constraint(y[u,i] <= x[i])

In [13]:
model.print_information()

Model: uflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 9
   - linear=9
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [14]:
type(y)

dict

In [15]:
model.add_constraint(sum(x) >= 1)

docplex.mp.LinearConstraint[](x_0+x_1+x_2,GE,1)

In [16]:
model.add_constraints(sum(y[u,i] for i in range(num_resources)) >= 1 for u in range(num_users))

[docplex.mp.LinearConstraint[](y_0_0+y_0_1+y_0_2,GE,1),
 docplex.mp.LinearConstraint[](y_1_0+y_1_1+y_1_2,GE,1),
 docplex.mp.LinearConstraint[](y_2_0+y_2_1+y_2_2,GE,1)]

In [17]:
model.print_information()

Model: uflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [18]:
resource_cost = sum(x[i] * z[i] for i in range(num_resources))
user_cost = sum(sum(y[u,i] * c[u][i] for i in range(num_resources)) for u in range(num_users))
print(resource_cost + user_cost)
model.minimize(resource_cost + user_cost)

12x_0+11x_1+13x_2+y_0_0+12y_0_1+3y_0_2+2y_1_0+7y_1_1+41y_1_2+19y_2_0+21y_2_1+7y_2_2


In [19]:
model.print_information()

Model: uflp
 - number of variables: 12
   - binary=12, integer=0, continuous=0
 - number of constraints: 13
   - linear=13
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [20]:
model.solve()

docplex.mp.solution.SolveSolution(obj=34,values={x_0:1,y_0_0:1,y_1_0:1,y..

In [21]:
model.print_solution()

objective: 34
status: OPTIMAL_SOLUTION(2)
  x_0=1
  y_0_0=1
  y_1_0=1
  y_2_0=1


# Single Source Capacitated Facility Location Problem (SSCFLP)

**Opis problema:**

Single Source Capacitated Facility Location Problem (SSCFLP) je varijacija problema lokacije resursa sa kapacitetima i ograničenjem da svaki korisnik koristi **tačno jedan resurs**. Problem se može opisati ovako:

- Imamo skup potencijalnih **resursa** i skup **korisnika**.  
- Svaka lokacija resursa ima **fiksni trošak izgradnje** i **kapacitet**.  
- Svaki korisnik ima **potražnju** koja mora biti zadovoljena iz samo jednog otvorenog resursa.  
- Između svakog korisnika i resursa postoji **trošak transporta**.  
- Cilj je odabrati koje resurse otvoriti i kako dodeliti korisnike tim resursima tako da **ukupni trošak (fiksna cena + trošak transporta) bude minimalan**, a kapaciteti resursa nisu prekoračeni.

**Formalna definicija:**

Označimo:  
- $R$ = skup svih potencijalnih resursa  
- $U$ = skup svih korisnika  
- $z_i$ = fiksna cena otvaranja resursa $i \in R$  
- $c_{ij}$ = trošak transporta korisnika $j \in U$ do resursa $i \in R$  
- $d_j$ = potražnja korisnika $j \in U$  
- $s_i$ = kapacitet resursa $i \in R$  
- $x_i \in \{0,1\}$ = 1 ako je resurs $i$ otvoren, 0 inače  
- $y_{ij} \in \{0,1\}$ = 1 ako korisnik $j$ koristi resurs $i$, 0 inače  

**Ciljna funkcija:**
$$
\min \sum_{i \in R} z_i x_i + \sum_{i \in R} \sum_{j \in U} c_{ij} y_{ij}
$$

**Ograničenja:**
1. Svaki korisnik mora biti dodeljen tačno jednom resursu:
$$
\sum_{i \in R} y_{ij} = 1 \quad \forall j \in U
$$
2. Kapacitet resursa ne sme biti prekoračen:
$$
\sum_{j \in U} d_j y_{ij} \le s_i \quad \forall i \in R
$$
3. Korisnik može biti dodeljen samo otvorenom resursu:
$$
y_{ij} \le x_i \quad \forall i \in R, \forall j \in U
$$
4. Promenljive su binarne:
$$
x_i \in \{0,1\}, \quad y_{ij} \in \{0,1\}
$$


In [22]:
model = Model(name='sscflp')

In [23]:
x = model.binary_var_list(num_resources, name='x')

In [24]:
y = model.binary_var_matrix(num_users, num_resources, name='y')

In [25]:
model.add_constraint(sum(x) >= 1)

docplex.mp.LinearConstraint[](x_0+x_1+x_2,GE,1)

In [26]:
for u in range(num_users):
    for i in range(num_resources):
        model.add_constraint(y[u,i] <= x[i])

In [27]:
model.add_constraints(sum(y[u,i] for i in range(num_resources)) >= 1 for u in range(num_users))

[docplex.mp.LinearConstraint[](y_0_0+y_0_1+y_0_2,GE,1),
 docplex.mp.LinearConstraint[](y_1_0+y_1_1+y_1_2,GE,1),
 docplex.mp.LinearConstraint[](y_2_0+y_2_1+y_2_2,GE,1)]

In [28]:
capacity = [2 for _ in range(num_resources)]
demand = [1.5 for _ in range(num_users)]

In [29]:
for i in range(num_resources):
    model.add_constraint(sum(y[u, i] * demand[u] for u in range(num_users)) <= capacity[i] * x[i])

In [30]:
resource_cost = sum(x[i] * z[i] for i in range(num_resources))
user_cost = sum(sum(y[u,i] * c[u][i] for i in range(num_resources)) for u in range(num_users))
print(resource_cost + user_cost)
model.minimize(resource_cost + user_cost)

12x_0+11x_1+13x_2+y_0_0+12y_0_1+3y_0_2+2y_1_0+7y_1_1+41y_1_2+19y_2_0+21y_2_1+7y_2_2


In [31]:
model.solve()

docplex.mp.solution.SolveSolution(obj=51,values={x_0:1,x_1:1,x_2:1,y_0_0..

In [32]:
model.print_solution()

objective: 51
status: OPTIMAL_SOLUTION(2)
  x_0=1
  x_1=1
  x_2=1
  y_0_0=1
  y_1_1=1
  y_2_2=1


# Multiple Source Capacitated Facility Location Problem (MSCFLP) sa realnim dodelama

**Opis problema:**

Multiple Source Capacitated Facility Location Problem (MSCFLP) sa realnim dodelama je varijacija problema lokacije resursa sa kapacitetima, gde svaki korisnik može imati **deo svoje potražnje zadovoljen iz jednog ili više resursa**. Problem se može opisati ovako:

- Imamo skup potencijalnih **resursa** i skup **korisnika**.  
- Svaka lokacija resursa ima **fiksni trošak izgradnje** i **kapacitet**.  
- Svaki korisnik ima **potražnju** koja mora biti zadovoljena kroz kombinaciju otvorenih resursa.  
- Između svakog korisnika i resursa postoji **trošak transporta**.  
- Cilj je odabrati koje resurse otvoriti i koliko dodeliti svakom korisniku tako da **ukupni trošak (fiksna cena + trošak transporta) bude minimalan**, a kapaciteti resursa nisu prekoračeni.

**Formalna definicija:**

Označimo:  
- $R$ = skup svih potencijalnih resursa  
- $U$ = skup svih korisnika  
- $z_i$ = fiksna cena otvaranja resursa $i \in R$  
- $c_{ij}$ = trošak transporta korisnika $j \in U$ do resursa $i \in R$  
- $p_j$ = potražnja korisnika $j \in U$  
- $s_i$ = kapacitet resursa $i \in R$  
- $x_i \in \{0,1\}$ = 1 ako je resurs $i$ otvoren, 0 inače  
- $y_{ij} \in [0,1]$ = deo potražnje korisnika $j$ koji se zadovoljava iz resursa $i$  

**Ciljna funkcija:**
$$
\min \sum_{i \in R} z_i x_i + \sum_{i \in R} \sum_{j \in U} c_{ij} y_{ij}
$$

**Ograničenja:**
1. Svaki korisnik mora imati **sveukupno zadovoljenu potražnju**:
$$
\sum_{i \in R} y_{ij} \ge 1 \quad \forall j \in U
$$
2. Kapacitet resursa ne sme biti prekoračen:
$$
\sum_{j \in U} d_j y_{ij} \le s_i \quad \forall i \in R
$$
3. Korisnik može koristiti resurs samo ako je otvoren:
$$
y_{ij} \le x_i \quad \forall i \in R, \forall j \in U
$$
4. Promenljive:
$$
x_i \in \{0,1\}, \quad y_{ij} \in [0,1]
$$

Jedina razlika u odnosu na prethodni SSCFLP model je u tipu promenljivih $y$ koje su sada realne, umesto binarnih.

In [33]:
model = Model(name='mscflp')

In [34]:
x = model.binary_var_list(num_resources, name='x')

In [35]:
y = model.continuous_var_matrix(num_users, num_resources, name='y')

In [36]:
model.add_constraint(sum(x) >= 1)

docplex.mp.LinearConstraint[](x_0+x_1+x_2,GE,1)

In [37]:
for u in range(num_users):
    for i in range(num_resources):
        model.add_constraint(y[u,i] <= x[i])

In [38]:
model.add_constraints(sum(y[u,i] for i in range(num_resources)) >= 1 for u in range(num_users))

[docplex.mp.LinearConstraint[](y_0_0+y_0_1+y_0_2,GE,1),
 docplex.mp.LinearConstraint[](y_1_0+y_1_1+y_1_2,GE,1),
 docplex.mp.LinearConstraint[](y_2_0+y_2_1+y_2_2,GE,1)]

In [39]:
for i in range(num_resources):
    model.add_constraint(sum(y[u, i] * demand[u] for u in range(num_users)) <= capacity[i] * x[i])

In [40]:
resource_cost = sum(x[i] * z[i] for i in range(num_resources))
user_cost = sum(sum(y[u,i] * c[u][i] for i in range(num_resources)) for u in range(num_users))
print(resource_cost + user_cost)
model.minimize(resource_cost + user_cost)

12x_0+11x_1+13x_2+y_0_0+12y_0_1+3y_0_2+2y_1_0+7y_1_1+41y_1_2+19y_2_0+21y_2_1+7y_2_2


In [41]:
model.solve()

docplex.mp.solution.SolveSolution(obj=48.3333,values={x_0:1,x_1:1,x_2:1,..

In [42]:
model.print_solution()

objective: 48.333
status: OPTIMAL_SOLUTION(2)
  x_0=1
  x_1=1
  x_2=1
  y_0_0=0.667
  y_0_2=0.333
  y_1_0=0.667
  y_1_1=0.333
  y_2_2=1.000
